In [1]:
#import libraries
from bs4 import BeautifulSoup as bs
import requests
import time
import datetime
import pandas as pd
import pprint
import re

import smtplib

In [2]:
#connect to website

URL = 'https://en.wikipedia.org/wiki/The_Lion_King'

headers = { "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.__ ) (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36 Edg/114.0.1823.__",  "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close",  "Upgrade-Insecure-Requests":"1"}

page = requests.get(URL, headers=headers)

Soup1 = bs(page.content,"html.parser")
soup2 = bs(Soup1.prettify(), 'html.parser')

title = soup2.find('th', class_="infobox-above summary").get_text()
info_box = soup2.find(class_="infobox vevent")

print(info_box)

<table class="infobox vevent">
<tbody>
<tr>
<th class="infobox-above summary" colspan="2" style="font-size: 125%; font-style: italic;">
             The Lion King
            </th>
</tr>
<tr>
<td class="infobox-image" colspan="2">
<a class="image" href="/wiki/File:The_Lion_King_poster.jpg" title="In an African savannah, several animals stare at a lion atop a tall rock. A lion's head can be seen in the clouds above. Atop the image is the text &quot;Walt Disney Pictures presents The Lion King&quot;.">
<img alt="In an African savannah, several animals stare at a lion atop a tall rock. A lion's head can be seen in the clouds above. Atop the image is the text &quot;Walt Disney Pictures presents The Lion King&quot;." class="thumbborder" data-file-height="384" data-file-width="259" decoding="async" height="326" src="//upload.wikimedia.org/wikipedia/en/thumb/3/3d/The_Lion_King_poster.jpg/220px-The_Lion_King_poster.jpg" srcset="//upload.wikimedia.org/wikipedia/en/3/3d/The_Lion_King_poster.jpg 1

In [3]:
title = soup2.find('th', class_="infobox-above summary").get_text()
title = title.strip()

print(title)

The Lion King


In [4]:
#extract all the rows
info_rows  = info_box.find_all("tr")
for row in info_rows:
    print(row)

<tr>
<th class="infobox-above summary" colspan="2" style="font-size: 125%; font-style: italic;">
             The Lion King
            </th>
</tr>
<tr>
<td class="infobox-image" colspan="2">
<a class="image" href="/wiki/File:The_Lion_King_poster.jpg" title="In an African savannah, several animals stare at a lion atop a tall rock. A lion's head can be seen in the clouds above. Atop the image is the text &quot;Walt Disney Pictures presents The Lion King&quot;.">
<img alt="In an African savannah, several animals stare at a lion atop a tall rock. A lion's head can be seen in the clouds above. Atop the image is the text &quot;Walt Disney Pictures presents The Lion King&quot;." class="thumbborder" data-file-height="384" data-file-width="259" decoding="async" height="326" src="//upload.wikimedia.org/wikipedia/en/thumb/3/3d/The_Lion_King_poster.jpg/220px-The_Lion_King_poster.jpg" srcset="//upload.wikimedia.org/wikipedia/en/3/3d/The_Lion_King_poster.jpg 1.5x" width="220"/>
</a>
<div class="inf

In [5]:
# Create an empty dictionary to store the data
movie_info ={}
# Function to extract actual content value
def actual_content_value(row_data):
    # Check if row has a list <li> in the HTML
    if row_data.find("li"):
        return [li.get_text(strip=True).replace('\n', ' ') for li in row_data.find_all("li")]
    else:
        #also cleaning the data
        return row_data.find("td").get_text(strip=True).replace('\n','').replace('\xa0', ' ').replace('\xa0', ' ')

for index, row in enumerate(info_rows):
    if index == 0:
        movie_info['title'] = row.find("th").get_text().strip()
    elif index == 1:
        continue
    else:
        content_key = row.find("th").get_text(" ",strip=True).replace('\n', ' ')
        content_value = actual_content_value(row)
        movie_info[content_key] = content_value





pprint.pprint(movie_info)

{'Box office': '$968.5 million[3]',
 'Budget': '$45 million[2]',
 'Country': 'United States',
 'Directed by': ['Roger Allers', 'Rob Minkoff'],
 'Distributed by': 'Buena Vista Pictures Distribution',
 'Edited by': 'Ivan Bilancio',
 'Language': 'English',
 'Music by': 'Hans Zimmer',
 'Produced by': 'Don Hahn',
 'Production companies': ['Walt Disney Pictures',
                          'Walt Disney Feature Animation'],
 'Release date': ['June\xa015,\xa01994(1994-06-15)(United States)'],
 'Running time': '88 minutes[1]',
 'Screenplay by': ['Irene Mecchi', 'Jonathan Roberts', 'Linda Woolverton'],
 'Starring': ['Jonathan Taylor Thomas',
              'Matthew Broderick',
              'James Earl Jones',
              'Jeremy Irons',
              'Moira Kelly',
              'Ernie Sabella',
              'Nathan Lane',
              'Robert Guillaume',
              'Rowan Atkinson',
              'Whoopi Goldberg',
              'Cheech Marin',
              'Jim Cummings',
              

In [6]:
#arranging the output
import pprint

# Create an empty dictionary to store the data
movie_info ={}
# Function to extract actual content value
def actual_content_value(row_data):
    # Check if row has a list <li> in the HTML
    if row_data.find("li"):
        return [li.get_text(strip=True).replace('\n', ' ') for li in row_data.find_all("li")]
    else:
        #also cleaning the data
        return row_data.find("td").get_text(strip=True).replace('\n','').replace('\xa0', ' ').replace('\xa0', ' ')

for index, row in enumerate(info_rows):
    if index == 0:
        movie_info['title'] = row.find("th").get_text().strip()
    elif index == 1:
        continue
    else:
        content_key = row.find("th").get_text(" ",strip=True).replace('\n', ' ')
        content_value = actual_content_value(row)
        movie_info[content_key] = content_value



# Replace '\xa0' with a space in the 'Release date' value
modified_release_date = movie_info['Release date'][0].replace('\xa0', ' ')

# Update the 'Release date' value in the dictionary
movie_info['Release date'] = [modified_release_date]

# Remove the space between the items in 'Production companies' list
production_companies = [company.replace(' ', '') for company in movie_info['Production companies']]

# Update the 'Production companies' value in the dictionary
movie_info['Production companies'] = production_companies


pprint.pprint(movie_info)

{'Box office': '$968.5 million[3]',
 'Budget': '$45 million[2]',
 'Country': 'United States',
 'Directed by': ['Roger Allers', 'Rob Minkoff'],
 'Distributed by': 'Buena Vista Pictures Distribution',
 'Edited by': 'Ivan Bilancio',
 'Language': 'English',
 'Music by': 'Hans Zimmer',
 'Produced by': 'Don Hahn',
 'Production companies': ['WaltDisneyPictures', 'WaltDisneyFeatureAnimation'],
 'Release date': ['June 15, 1994(1994-06-15)(United States)'],
 'Running time': '88 minutes[1]',
 'Screenplay by': ['Irene Mecchi', 'Jonathan Roberts', 'Linda Woolverton'],
 'Starring': ['Jonathan Taylor Thomas',
              'Matthew Broderick',
              'James Earl Jones',
              'Jeremy Irons',
              'Moira Kelly',
              'Ernie Sabella',
              'Nathan Lane',
              'Robert Guillaume',
              'Rowan Atkinson',
              'Whoopi Goldberg',
              'Cheech Marin',
              'Jim Cummings',
              'Madge Sinclair'],
 'Story by': ['Bar

In [7]:
page = requests.get("https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films")

#convert to beautiful soup object
soup = bs(page.content)

#print out the HTML
contents = soup.prettify()
print(contents)

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-enabled vector-feature-main-menu-pinned-disabled vector-feature-limited-width-enabled vector-feature-limited-width-content-enabled vector-feature-zebra-design-disabled" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of Walt Disney Pictures films - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-enabled vector-feature-main-menu-pinned-disabled vector-feature-limited-width-enabled vector-feature-limited-width-content-enabled vector-feature-zebra-design-disabled";(function(){var cookie=document.c

In [8]:
all_td_elements = soup.find_all('td')

for td in all_td_elements:
    link_element = td.find('a')

    if link_element:
        title = link_element.get('title')
        href = link_element.get('href')

        if title and title != 'None'and title != '20th Digital Studio' and not title.startswith('List')and not title.startswith('Walt'):
            print("Title:", title) 
            print("Link:", href)

Title: Snow White and the Seven Dwarfs (1937 film)
Link: /wiki/Snow_White_and_the_Seven_Dwarfs_(1937_film)
Title: RKO Pictures
Link: /wiki/RKO_Pictures
Title: Pinocchio (1940 film)
Link: /wiki/Pinocchio_(1940_film)
Title: Fantasia (1940 film)
Link: /wiki/Fantasia_(1940_film)
Title: The Reluctant Dragon (1941 film)
Link: /wiki/The_Reluctant_Dragon_(1941_film)
Title: Dumbo
Link: /wiki/Dumbo
Title: Bambi
Link: /wiki/Bambi
Title: Saludos Amigos
Link: /wiki/Saludos_Amigos
Title: Victory Through Air Power (film)
Link: /wiki/Victory_Through_Air_Power_(film)
Title: United Artists
Link: /wiki/United_Artists
Title: The Three Caballeros
Link: /wiki/The_Three_Caballeros
Title: Make Mine Music
Link: /wiki/Make_Mine_Music
Title: Song of the South
Link: /wiki/Song_of_the_South
Title: Fun and Fancy Free
Link: /wiki/Fun_and_Fancy_Free
Title: Melody Time
Link: /wiki/Melody_Time
Title: So Dear to My Heart
Link: /wiki/So_Dear_to_My_Heart
Title: The Adventures of Ichabod and Mr. Toad
Link: /wiki/The_Advent

In [9]:
all_td_elements = soup.find_all('td')
distinct_titles = set()  # Create a set to store distinct titles
 
for td in all_td_elements:
    link_element = td.find('a')

    if link_element:
        title = link_element.get('title')

        if title and title != 'None' and title != '20th Digital Studio' and not title.startswith('List') and not title.startswith('Walt'):
            print("Title:", title)
            distinct_titles.add(title)  # Add title to the set

# Calculate the sum of distinct titles
title_sum = len(distinct_titles)

print("Sum of distinct titles:", title_sum)

Title: Snow White and the Seven Dwarfs (1937 film)
Title: RKO Pictures
Title: Pinocchio (1940 film)
Title: Fantasia (1940 film)
Title: The Reluctant Dragon (1941 film)
Title: Dumbo
Title: Bambi
Title: Saludos Amigos
Title: Victory Through Air Power (film)
Title: United Artists
Title: The Three Caballeros
Title: Make Mine Music
Title: Song of the South
Title: Fun and Fancy Free
Title: Melody Time
Title: So Dear to My Heart
Title: The Adventures of Ichabod and Mr. Toad
Title: Cinderella (1950 film)
Title: Treasure Island (1950 film)
Title: Alice in Wonderland (1951 film)
Title: The Story of Robin Hood (film)
Title: Peter Pan (1953 film)
Title: The Sword and the Rose
Title: The Living Desert
Title: Buena Vista Pictures Distribution
Title: Rob Roy: The Highland Rogue
Title: RKO Pictures
Title: The Vanishing Prairie
Title: 20,000 Leagues Under the Sea (1954 film)
Title: Davy Crockett: King of the Wild Frontier (film)
Title: Disney anthology television series
Title: Lady and the Tramp
Title:

In [10]:
#filter the data to remove the undesired titles
filtered_td_elements = []

for td in all_td_elements:
    link_element = td.find('a')

    if link_element:
        title = link_element.get('title')

        if title and title != 'None' and title != '20th Digital Studio' and not title.startswith('List') and not title.startswith('Walt'):
            filtered_td_elements.append(td)

count = len(filtered_td_elements)
print("Count:", count)

Count: 891


In [11]:
#checking for a certain title
#desired_title = "The Princess and the Frog"
#found = False

#for td in td_elements:
    #     title = link_element.get('title')

       # if title and title == desired_title:
            ##href = link_element.get('href')
            #print("Title:", title)
           # print("Link:", href)

#if not found:
    #print("The title", desired_title, "was not found in the data.")

In [12]:
# Function to extract actual content value
def actual_content_value(row_data):
    # Remove all sup tags
    for sup in row_data.find_all("sup"):
        sup.decompose()
    # Check if row has a list <li> in the HTML
    if row_data.find("li"):
        return [li.get_text(strip=True).replace('\n', ' ') for li in row_data.find_all("li")]
    else:
        return row_data.find("td").get_text(strip=True).replace('\n', '').replace('\xa0', ' ')
    

def get_info_box(url):
    page = requests.get(url)

    # convert to beautiful soup object
    soup = bs(page.content)

    # extract all the rows
    info_box = soup.find(class_="infobox vevent")
    info_rows = info_box.find_all("tr")
   
    
    movie_info = {}  # Create an empty dictionary to store the data

    for index, row in enumerate(info_rows):
        if index == 0:
            movie_info['title'] = row.find("th").get_text().strip()
        elif index == 1:
            continue
        else:
            content_key = row.find("th").get_text(" ", strip=True).replace('\n', ' ')
            content_value = actual_content_value(row)
            if content_value is not None:
                movie_info[content_key] = content_value

            # Replace '\xa0' with a space in the release date value
    release_date_key = 'Release date' if 'Release date' in movie_info else 'Release dates'
    modified_release_date = movie_info.get(release_date_key, [''])[0].replace('\xa0', ' ')

    # Update the release date value in the dictionary
    movie_info[release_date_key] = [modified_release_date]

    # Check if the key is 'Production company' or 'Production companies'
    production_key = 'Production company' if 'Production company' in movie_info else 'Production companies'

    # Remove the space between the items in 'Production companies' list
    production_value = movie_info.get(production_key, [])
    if isinstance(production_value, list):
        production_companies = [company.replace(' ', '') for company in production_value]
        movie_info[production_key] = production_companies

    pprint.pprint(movie_info)
    return movie_info

    

In [13]:
def get_info_box(url):
    page = requests.get(url)
    soup = bs(page.content)


    # Function to extract actual content value
    def actual_content_value(row_data):
    #Data cleaning# Remove all sup tags
        for sup in row_data.find_all("sup"):
            sup.decompose()
    #Data Cleaning #Check if row has a list <li> in the HTML
        if row_data.find("li"):
            return [li.get_text(strip=True).replace('\n', ' ') for li in row_data.find_all("li")]
        else:
            return row_data.find("td").get_text(strip=True).replace('\n', '').replace('\xa0', ' ')
        
    def convert_value_to_int(value):
        value = value.replace('$', '').replace(',', '').replace('>', '').replace('<', '').strip()
        
                # Remove references
        value = re.sub(r'\[\d+\]', '', value)

        suffixes = {
            'million': 1_000_000,
            'billion': 1_000_000_000,
            'trillion': 1_000_000_000_000
        }

        # Check for a range of values
        if '–' in value:
            values = value.split('–')
            upper_value = extract_numeric_value(values[1])
            suffix = extract_suffix(values[1])
        else:
            upper_value = extract_numeric_value(value)
            suffix = extract_suffix(value)

        if upper_value is None:
            return None

        if suffix in suffixes:
            return int(upper_value * suffixes[suffix])
        else:
            return int(upper_value)


    def extract_numeric_value(value):
        # Extract numeric value from a string
        pattern = r'(\d+(\.\d+)?)'
        match = re.search(pattern, value)
        if match:
            return float(match.group(1))
        else:
            return None


    def extract_suffix(value):
        # Extract suffix from a string
        suffixes = ['million', 'billion', 'trillion']
        for suffix in suffixes:
            if suffix in value:
                return suffix
        return ''  # Empty suffix if not found

    def parse_running_time(time_str):
        # Extract the numeric value from the time string
        duration = re.findall(r'\d+', time_str)

        if len(duration) > 0:
            # Assuming the numeric value represents minutes
            minutes = int(duration[0])

            # Create a timedelta object with the minutes duration
            time_delta = datetime.timedelta(minutes=minutes)

            # Format the time duration as HH:MM
            time_format = "{:02d}:{:02d}".format(time_delta.seconds // 3600, (time_delta.seconds // 60) % 60)

            return time_format

        return None
    
    def convert_to_date(date_string):
        date_string = date_string.split('(')[0].strip()
        release_date = datetime.datetime.strptime(date_string, '%B %d, %Y').date()
        return release_date.strftime('%Y-%m-%d')
        
    info_box = soup.find(class_="infobox vevent")
    info_rows = info_box.find_all("tr")

    movie_info = {}

    for index, row in enumerate(info_rows):
        if index == 0:
            movie_info['title'] = row.find("th").get_text().strip()
        elif index == 1:
            continue
        else:
            content_key = row.find("th").get_text(" ", strip=True).replace('\n', ' ')
            content_value = actual_content_value(row)
            if content_value is not None:
                movie_info[content_key] = content_value
                
    
    release_date_key = 'Release date' if 'Release date' in movie_info else 'Release dates'
    modified_release_date = movie_info.get(release_date_key, [''])[0].split('(')[0].strip()
    converted_release_date = convert_to_date(modified_release_date)
    movie_info[release_date_key] = [converted_release_date]



    production_key = 'Production company' if 'Production company' in movie_info else 'Production companies'
    production_value = movie_info.get(production_key, [])
    if isinstance(production_value, list):
        production_companies = [company.replace(' ', '') for company in production_value]
        movie_info[production_key] = production_companies
    
    box_office_key = 'Box office'
    if box_office_key in movie_info:
        box_office_value = movie_info[box_office_key]
        movie_info[box_office_key] = convert_value_to_int(box_office_value)

    budget_key = 'Budget'
    if budget_key in movie_info:
        budget_value = movie_info[budget_key]
        movie_info[budget_key] = convert_value_to_int(budget_value)

    
    running_time_key = 'Running time'
    if running_time_key in movie_info:
        running_time_value = movie_info[running_time_key]
        movie_info[running_time_key] = parse_running_time(running_time_value)

    
    
    pprint.pprint(movie_info)
    return movie_info

In [ ]:
#PUT IT ALL TOGETHER

In [14]:
page = requests.get("https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films")
soup = bs(page.content)
all_td_elements = soup.find_all('td')

distinct_titles = set()  # Create a set to store distinct titles
 
for td in all_td_elements:
    link_element = td.find('a')

    if link_element:
        title = link_element.get('title')

        if title and title != 'None' and title != '20th Digital Studio' and not title.startswith('List') and not title.startswith('Walt'):
            distinct_titles.add(title)  # Add title to the set

# Calculate the sum of distinct titles
title_sum = len(distinct_titles)



base_path = "https://www.wikipedia.org/" 

movie_info_list = []
count = 0 # Counter variable to track the number of titles

for td in all_td_elements:
    link_element = td.find('a')

    if link_element:
        title = link_element.get('title')
        href = link_element.get('href')

        if title and title != 'None' and title != '20th Digital Studio' and not title.startswith('List') and not title.startswith('Walt'):
            print("Title:", title) 
            print("Link:", href)
            try:
                relative_path = href
                full_path = base_path + relative_path
                
                    
                   # Append the scraped data to the list  
                movie_info_list.append(get_info_box(full_path))
                count += 1  # Increment the count variable
                
                if count >= title_sum:
                    break

            except Exception as e:
                print(href)
                print(e)
    if count >= title_sum:
         break    

Title: Snow White and the Seven Dwarfs (1937 film)
Link: /wiki/Snow_White_and_the_Seven_Dwarfs_(1937_film)
{'Based on': 'Snow Whiteby theBrothers Grimm',
 'Box office': 418000000,
 'Budget': 1500000,
 'Country': 'United States',
 'Directed by': ['David Hand',
                 'Perce Pearce',
                 'William Cottrell',
                 'Larry Morey',
                 'Wilfred Jackson',
                 'Ben Sharpsteen'],
 'Distributed by': 'RKO Radio Pictures',
 'Language': 'English',
 'Music by': ['Frank Churchill', 'Leigh Harline', 'Paul Smith'],
 'Produced by': 'Walt Disney',
 'Production company': 'Walt Disney Productions',
 'Release dates': ['1937-12-21'],
 'Running time': '01:23',
 'Starring': ['Adriana Caselotti',
              'Roy Atwell',
              'Pinto Colvig',
              'Otis Harlan',
              'Scotty Mattraw',
              'Billy Gilbert',
              'Eddie Collins'],
 'Story by': ['Ted Sears',
              'Richard Creedon',
              'Ott

In [15]:
movie_info_list

[{'title': 'Snow White and the Seven Dwarfs',
  'Directed by': ['David Hand',
   'Perce Pearce',
   'William Cottrell',
   'Larry Morey',
   'Wilfred Jackson',
   'Ben Sharpsteen'],
  'Story by': ['Ted Sears',
   'Richard Creedon',
   'Otto Englander',
   'Dick Rickard',
   'Earl Hurd',
   'Merrill De Maris',
   'Dorothy Ann Blank',
   'Webb Smith'],
  'Based on': 'Snow Whiteby theBrothers Grimm',
  'Produced by': 'Walt Disney',
  'Starring': ['Adriana Caselotti',
   'Roy Atwell',
   'Pinto Colvig',
   'Otis Harlan',
   'Scotty Mattraw',
   'Billy Gilbert',
   'Eddie Collins'],
  'Music by': ['Frank Churchill', 'Leigh Harline', 'Paul Smith'],
  'Production company': 'Walt Disney Productions',
  'Distributed by': 'RKO Radio Pictures',
  'Release dates': ['1937-12-21'],
  'Running time': '01:23',
  'Country': 'United States',
  'Language': 'English',
  'Budget': 1500000,
  'Box office': 418000000},
 {'title': 'Pinocchio',
  'Directed by': ['Ben Sharpsteen',
   'Hamilton Luske',
   'Bill 

In [16]:
df = pd.DataFrame(movie_info_list)

# Specify the file path
file_path = r"\\gdl\shareddata\Users\mj\Desktop\project-blog\movie_info_List.xlsx"

# Export the DataFrame to an Excel file at the specified path
df.to_excel(file_path, index=False)